In [1]:
import sys
print(sys.executable)
print(sys.version)

/Library/Developer/CommandLineTools/usr/bin/python3
3.9.6 (default, Oct 17 2025, 17:15:53) 
[Clang 17.0.0 (clang-1700.4.4.1)]


In [2]:
pip install tensorflow-macos==2.15.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow-metal==1.1.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
## 4 classes
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator

warnings.filterwarnings("ignore")

train_dir = "../week3/alzheimer dataset/train"
test_dir  = "../week3/alzheimer dataset/test"


## object for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale= 1/.255, ##normalize pixel values
    shear_range = 0.4 , ## slant range, titling medical images might harm its quality
    zoom_range = 0.2 , ## zoom range for scale 
    width_shift_range = 0.2 , ##moving image left to right
    height_shift_range = 0.2, ##moving image top to bottom
    rotation_range = 90 , ## this might harm medical image classification
    horizontal_flip =True,
    fill_mode = 'nearest'
)

test_datagen = ImageDataGenerator(rescale =1/.255)

##load the images ,labels , ensuring grayscale (1 color channel)
train_generator = train_datagen.flow_from_directory(
    train_dir ,
    target_size =(128, 128), ##resize the images 
    color_mode ='grayscale',
    class_mode='categorical',
    
) # default batch size =32

test_generator =test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    color_mode ='grayscale',
    class_mode='categorical'
)


/Users/sheenachanda/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Found 4649 images belonging to 4 classes.
Found 1813 images belonging to 4 classes.


Import Pretrained Model  -VGG16

In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input , Concatenate ,Flatten ,Dense , Dropout
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [6]:
input_tensor= Input(shape=(128,128,1))
input_tensor_rgb = Concatenate()([input_tensor, input_tensor,input_tensor])

vgg16=VGG16(
    weights='imagenet',
    include_top=False,
    input_tensor=input_tensor_rgb
)

2026-02-16 19:38:05.461500: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2026-02-16 19:38:05.461525: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2026-02-16 19:38:05.461536: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.88 GB
2026-02-16 19:38:05.461568: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-16 19:38:05.461582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


58889256/58889256 [==============================] - 83s 1us/step


In [7]:
 

## create a new model and add VGG model

model_vgg16 = Sequential()
model_vgg16.add(vgg16)

## add custom classification head 
model_vgg16.add(Flatten())
model_vgg16.add(Dense(256 , activation = 'relu'))
model_vgg16.add(Dropout(0.5))
model_vgg16.add(Dense(128 , activation = 'relu'))
model_vgg16.add(Dropout(0.5))

#Final output for 4 classes
model_vgg16.add(Dense(4, activation='softmax'))
##Freeze the vgg16 layers (for transfer learning)
for layer in vgg16.layers:
    layer.trainable =False

##compile the model
model_vgg16.compile(
    optimizer= Adam(learning_rate = 0.01),
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

model_vgg16.summary()


                

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 5

In [8]:
model_vgg16.fit(train_generator, epochs=50)

Epoch 1/50


2026-02-16 19:39:29.800632: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2026-02-16 19:39:29.825696: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


146/146 [==============================] - 11s 56ms/step - loss: 4829.7812 - accuracy: 0.4438
Epoch 2/50
146/146 [==============================] - 7s 51ms/step - loss: 3486.6414 - accuracy: 0.4481
Epoch 3/50
146/146 [==============================] - 7s 51ms/step - loss: 3619.4580 - accuracy: 0.4539
Epoch 4/50
146/146 [==============================] - 7s 50ms/step - loss: 4479.3760 - accuracy: 0.4620
Epoch 5/50
146/146 [==============================] - 7s 51ms/step - loss: 4404.8823 - accuracy: 0.4558
Epoch 6/50
146/146 [==============================] - 7s 51ms/step - loss: 5578.3799 - accuracy: 0.4597
Epoch 7/50
146/146 [==============================] - 7s 51ms/step - loss: 4027.2610 - accuracy: 0.4579
Epoch 8/50
146/146 [==============================] - 8s 51ms/step - loss: 4839.4556 - accuracy: 0.4610
Epoch 9/50
146/146 [==============================] - 8s 52ms/step - loss: 4848.9497 - accuracy: 0.4631
Epoch 10/50
146/146 [==============================] - 8s 53ms/step - loss

In [9]:
loss,accuracy_vgg=model_vgg16.evaluate(test_generator)
print('Test accuracy with pretrained weights from vgg16',accuracy_vgg)

57/57 [==============================] - 4s 62ms/step - loss: 25167.2812 - accuracy: 0.2212
Test accuracy with pretrained weights from vgg16 0.22118036448955536


VGG16 Variant2 -custom vgg18 , conv2d layers

In [13]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input , Concatenate ,Flatten ,Dense , Dropout,Conv2D,MaxPooling2D,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow as tf

In [14]:
input_tensor= Input(shape=(128,128,1))
input_tensor_rgb = Concatenate()([input_tensor, input_tensor,input_tensor])

vgg16=VGG16(
    weights='imagenet',
    include_top=False,
    input_tensor=input_tensor_rgb
)
vgg16.trainable=False
 

## create a new model and add VGG model

model_vgg16_plus = Sequential()
model_vgg16_plus.add(vgg16)
##add custom conv layers
model_vgg16_plus.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model_vgg16_plus.add(BatchNormalization())
model_vgg16_plus.add(MaxPooling2D(2,2))

model_vgg16_plus.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model_vgg16_plus.add(BatchNormalization())
model_vgg16_plus.add(MaxPooling2D(2,2))


## add custom classification head 
model_vgg16_plus.add(Flatten())
##or model_vgg16_plus.add(GlobalAveragePooling2D())
##Dense Layers
model_vgg16_plus.add(Dense(256 , activation = 'relu'))
model_vgg16_plus.add(Dropout(0.5))
model_vgg16_plus.add(Dense(128 , activation = 'relu'))
model_vgg16_plus.add(Dropout(0.5))

#Final output for 4 classes
model_vgg16_plus.add(Dense(4, activation='softmax'))
##Freeze the vgg16 layers (for transfer learning)
# for layer in vgg16.layers:
#     layer.trainable =False

##compile the model
model_vgg16_plus.compile(
    optimizer= Adam(learning_rate = 0.01),
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

model_vgg16_plus.summary()


                

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         589952    
                                                                 
 batch_normalization_2 (Bat  (None, 4, 4, 128)         512       
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 2, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 128)         147584    
                                                                 
 batch_normalization_3 (Bat  (None, 2, 2, 128)        

In [15]:
model_vgg16_plus.fit(train_generator, epochs=50)

Epoch 1/50
146/146 [==============================] - 11s 61ms/step - loss: 12.7371 - accuracy: 0.4612
Epoch 2/50
146/146 [==============================] - 8s 54ms/step - loss: 12.0843 - accuracy: 0.4612
Epoch 3/50
146/146 [==============================] - 8s 53ms/step - loss: 14.4178 - accuracy: 0.4573
Epoch 4/50
146/146 [==============================] - 8s 53ms/step - loss: 14.9818 - accuracy: 0.4562
Epoch 5/50
146/146 [==============================] - 8s 53ms/step - loss: 12.1329 - accuracy: 0.4564
Epoch 6/50
146/146 [==============================] - 8s 53ms/step - loss: 12.4954 - accuracy: 0.4597
Epoch 7/50
146/146 [==============================] - 8s 52ms/step - loss: 12.6240 - accuracy: 0.4493
Epoch 8/50
146/146 [==============================] - 8s 52ms/step - loss: 12.8324 - accuracy: 0.4558
Epoch 9/50
146/146 [==============================] - 8s 52ms/step - loss: 10.8932 - accuracy: 0.4638
Epoch 10/50
146/146 [==============================] - 8s 52ms/step - loss: 11.06

In [17]:
loss,accuracy_vgg=model_vgg16_plus.evaluate(test_generator)
print('Test accuracy with pretrained weights from vgg16 + custom conv layers',accuracy_vgg)

57/57 [==============================] - 3s 51ms/step - loss: 47.0348 - accuracy: 0.2438
Test accuracy with pretrained weights from vgg16 + custom conv layers 0.24379481375217438


VGG19 model

In [18]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input , Concatenate ,Flatten ,Dense , Dropout,Conv2D,MaxPooling2D,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow as tf

In [19]:
input_tensor= Input(shape=(128,128,1))
input_tensor_rgb = Concatenate()([input_tensor, input_tensor,input_tensor])

vgg19=VGG19(
    weights='imagenet',
    include_top=False,
    input_tensor=input_tensor_rgb
)
vgg19.trainable=False
 

## create a new model and add VGG model

model_vgg19_plus = Sequential()
model_vgg19_plus.add(vgg19)
##add custom conv layers
model_vgg19_plus.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model_vgg19_plus.add(BatchNormalization())
model_vgg19_plus.add(MaxPooling2D(2,2))

model_vgg19_plus.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model_vgg19_plus.add(BatchNormalization())
model_vgg19_plus.add(MaxPooling2D(2,2))


## add custom classification head 
model_vgg19_plus.add(Flatten())
##or model_vgg16_plus.add(GlobalAveragePooling2D())
##Dense Layers
model_vgg19_plus.add(Dense(256 , activation = 'relu'))
model_vgg19_plus.add(Dropout(0.5))
model_vgg19_plus.add(Dense(128 , activation = 'relu'))
model_vgg19_plus.add(Dropout(0.5))

#Final output for 4 classes
model_vgg19_plus.add(Dense(4, activation='softmax'))
##Freeze the vgg16 layers (for transfer learning)
# for layer in vgg16.layers:
#     layer.trainable =False

##compile the model
model_vgg19_plus.compile(
    optimizer= Adam(learning_rate = 0.01),
    loss= 'categorical_crossentropy',
    metrics=['accuracy']
)

model_vgg19_plus.summary()


                

80134624/80134624 [==============================] - 94s 1us/step
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 conv2d_4 (Conv2D)           (None, 4, 4, 128)         589952    
                                                                 
 batch_normalization_4 (Bat  (None, 4, 4, 128)         512       
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 2, 2, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 2, 2, 128)         147584    
                                                      

In [20]:
model_vgg19_plus.fit(train_generator, epochs=50)

Epoch 1/50
146/146 [==============================] - 10s 64ms/step - loss: 12.4823 - accuracy: 0.4549
Epoch 2/50
146/146 [==============================] - 9s 64ms/step - loss: 11.1099 - accuracy: 0.4663
Epoch 3/50
146/146 [==============================] - 9s 63ms/step - loss: 10.5399 - accuracy: 0.4633
Epoch 4/50
146/146 [==============================] - 9s 63ms/step - loss: 8.1569 - accuracy: 0.4659
Epoch 5/50
146/146 [==============================] - 9s 64ms/step - loss: 8.4276 - accuracy: 0.4709
Epoch 6/50
146/146 [==============================] - 9s 65ms/step - loss: 8.6183 - accuracy: 0.4676
Epoch 7/50
146/146 [==============================] - 9s 65ms/step - loss: 10.0380 - accuracy: 0.4691
Epoch 8/50
146/146 [==============================] - 10s 67ms/step - loss: 10.5075 - accuracy: 0.4543
Epoch 9/50
146/146 [==============================] - 10s 66ms/step - loss: 11.1681 - accuracy: 0.4582
Epoch 10/50
146/146 [==============================] - 10s 69ms/step - loss: 10.03

In [21]:
loss,accuracy_vgg=model_vgg19_plus.evaluate(test_generator)
print('Test accuracy with pretrained weights from vgg19 + custom conv layers',accuracy_vgg)

57/57 [==============================] - 6s 98ms/step - loss: 29.3551 - accuracy: 0.2438
Test accuracy with pretrained weights from vgg19 + custom conv layers 0.24379481375217438
